# Exploración visual de datos

El análisis exploratorio de datos (**EDA**) es el proceso de realizar consultas inciales de los datos en un esfuerzo por descubrir patrones, probar hipótesis, resumir características importantes de los datos, e identificar potenciales _outliers_ problemáticos. Este proceso se conduce principalmente por la producción de estadísticos y exploración visual de los datos.

La expansión espacial del EDA es conocida como análisis exploratorio de datos espaciales (**ESDA**), y es una etapa crítica en el _pipeline_ de la ciencia de datos espacial. Es una expansión en el sentido de que el enfoque de un EDA está en entender la correlación entre variables, mientras que ESDA se basa en entender la correlación de las variables con respecto a su ubicación en el espacio.

El primero paso en ESDA es la **cartografía**, o el proceso de creación de mapas. La cartografía es en sí mismo un arte en un sentido en el que toma datos y los representa visualmente de modo que personas tanto técnicas como no-técnicas entiendan y encuentren sentido en los datos; y es una ciencia en un sentido en el que los mapas deben ser derivados de los datos y ser representativos de ellos.

Cabe destacar que no es una práctica común en un EDA convencional, que típicamente se enfoca en las siguientes tareas:

* Estadísticos básicos de los datos
* Entender la distribución de los datos
* Gestión de valores faltantes
* Identificación de _outliers_
* Entender la correlación entre variables

Para ilustrarlo, se contrastan las diferencias de ambos enfoques en un ejemplo que utiliza datos de las listas de **Airbnb de Nueva York** en dos secciones:

* [Análisis exploratorio de datos](#Análisis-exploratorio-de-datos)
* [Análisis exploratorio de datos espaciales](#Análisis-exploratorio-de-datos-espaciales)

Es importante mencionar que los datos son proporcionados por [Inside Airbnb](http://insideairbnb.com/get-the-data/), y se puede consultar la información de múltiples ciudades

Carga de librerías

In [1]:
import os
os.environ['USE_PYGEOS'] = '0'

import pysal
import splot
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

Ruta relativa a archivo de datos.

Siempre que se carga un archivo de datos es necesario consultar su directorio de datos, o en caso de no contar con ello sería necesario construir uno para mayor entendimiento de los mismos. En el caso de los datos de _Inside Airbnb_ se cuenta con un directorio en la siguiente [liga](https://docs.google.com/spreadsheets/d/1iWCNJcSutYqpULSQHlNyGInUvHg2BoUGoNRIGa6Szc4/edit#gid=982310896))6)

In [2]:
ROOT = os.path.dirname(os.getcwd())
data_path = os.path.join(ROOT, 'data', 'Data pt1')

# Reading in the data - REMINDER: You'll need to download this data yourself from InsideAirbnb
listings = pd.read_csv(
    os.path.join(data_path, 'listings.csv.gz')
    ,compression='gzip')

# Información general
listings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43566 entries, 0 to 43565
Data columns (total 75 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   id                                            43566 non-null  int64  
 1   listing_url                                   43566 non-null  object 
 2   scrape_id                                     43566 non-null  int64  
 3   last_scraped                                  43566 non-null  object 
 4   source                                        43566 non-null  object 
 5   name                                          43566 non-null  object 
 6   description                                   42866 non-null  object 
 7   neighborhood_overview                         25201 non-null  object 
 8   picture_url                                   43566 non-null  object 
 9   host_id                                       43566 non-null 

# Análisis exploratorio de datos

In [ ]:
# Subsetting the data

listings_sub = listings[['id','property_type','neighbourhood_cleansed', 'neighbourhood_group_cleansed','beds','bathrooms', 'price','latitude','longitude']]

listings_sub.head()

In [ ]:
listings_sub.info()

In [ ]:
# Cleaning the data
listings_sub.drop(columns=['beds','bathrooms'], inplace=True)
listings_sub["price"] = listings_sub["price"].replace("[$,]", "", regex=True).astype(float)

In [ ]:
listings_sub

In [ ]:
listings_sub['price'].describe()

In [ ]:
%matplotlib inline
plt.hist(listings_sub['price'], bins=25, density=True, alpha=0.6)
plt.show()

In [ ]:
import seaborn as sns
sns.distplot(listings_sub['price'], kde=True)
plt.show()

# Análisis exploratorio de datos espaciales

In [ ]:
# Convert the pandas dataframe to a geopandas dataframe
listings_sub_gpd = gpd.GeoDataFrame(listings_sub, geometry=gpd.points_from_xy(listings_sub.longitude, listings_sub.latitude, crs=4326))

In [ ]:
import geoplot.crs as gcrs
import geoplot as gplt
ax = gplt.webmap(listings_sub_gpd,projection=gcrs.WebMercator())
gplt.pointplot(listings_sub_gpd, ax=ax)

In [ ]:
# Reading in the New York Census Tracts
NY_tracts_path = "https://www2.census.gov/geo/tiger/TIGER2021/TRACT/tl_2021_36_tract.zip"
NY_Tracts = gpd.read_file(NY_tracts_path)
NY_Tracts = NY_Tracts.to_crs(4326)

# Subsetting the census tracts to those in the New York CBSA
cbsa_path = 'https://www2.census.gov/geo/tiger/TIGER2021/CBSA/tl_2021_us_cbsa.zip'
cbsas = gpd.read_file(cbsa_path)
NY_cbsa = cbsas[cbsas['GEOID']=='35620']

In [ ]:
# Borough Boundaries
Boroughs = gpd.read_file(data_path + r"NYC Boroughs\nybb_22a\nybb.shp")
Boroughs = Boroughs.to_crs(4326)
Boroughs.plot()

In [ ]:
# Creating a heatmap of Airbnb locations

# Creating a heatmap raster from the points
ax = gplt.kdeplot(
    listings_sub_gpd,shade=True, cmap='Reds',
    clip=Boroughs.geometry,
    projection=gcrs.WebMercator())

# Plotting the heatmap on top of the boroughs for context
gplt.polyplot(Boroughs, ax=ax, zorder=1)

In [ ]:
# Reading in the New York Census Tracts
NY_tracts_path = "https://www2.census.gov/geo/tiger/TIGER2021/TRACT/tl_2021_36_tract.zip"
NY_Tracts = gpd.read_file(NY_tracts_path)
NY_Tracts = NY_Tracts.to_crs(4326)

# Subsetting the census tracts to those in the New York CBSA
cbsa_path = 'https://www2.census.gov/geo/tiger/TIGER2021/CBSA/tl_2021_us_cbsa.zip'
cbsas = gpd.read_file(cbsa_path)
NY_cbsa = cbsas[cbsas['GEOID']=='35620']

mask = NY_Tracts.intersects(NY_cbsa.loc[620,'geometry'])
NY_Tracts_subset = NY_Tracts.loc[mask]

# Aggregating the airbnb locations to the NY census tracts
NY_Tracts_sj = gpd.sjoin(NY_Tracts_subset, listings_sub_gpd, how='left', op='contains')
NY_Tracts_sj = NY_Tracts_sj[['GEOID','price','geometry']]
NY_Tracts_Agg = NY_Tracts_sj.dissolve(by='GEOID', aggfunc='mean')

NY_Tracts_Agg

In [ ]:
gplt.choropleth(NY_Tracts_Agg, hue="price", cmap="Greens", figsize=(60,30), legend=True)

In [ ]:
import geoviews
geoviews.extension("bokeh")

choropleth = geoviews.Polygons(data=NY_Tracts_Agg, vdims=["price","GEOID"])

choropleth.opts(height=600, width=900, title="NYC Airbnb Price",
                tools=["hover"], cmap="Greens", colorbar=True, colorbar_position="bottom")

In [ ]:
import statistics

# Calculating the mean and standard deviation
mean_price = statistics.mean(NY_Tracts_Agg['price'].dropna())
stdev = statistics.stdev(NY_Tracts_Agg['price'].dropna())

print("The mean Airbnb price is: % s " % (round(mean_price, 2)))
print("The standard deviation of Airbnb prices is: % s " % (round(stdev, 2)))

# Droping records that are more than 1 standard deviation from the mean
NY_Tracts_Agg_filtered = NY_Tracts_Agg[NY_Tracts_Agg['price'] < mean_price+stdev]

choropleth = geoviews.Polygons(data=NY_Tracts_Agg_filtered, vdims=["price","GEOID"])

choropleth.opts(height=600, width=900, title="NYC Airbnb Price",
                tools=["hover"], cmap="Greens", colorbar=True, colorbar_position="bottom")